In [ ]:
from hydra import compose, initialize
from omegaconf import OmegaConf
initialize(config_path="../../configs", version_base="1.3")
cfg = compose(config_name="eval_sim")

In [ ]:
import torch
import numpy as np
from open_anything_diffusion.models.modules.dit_models import DGDiT
from open_anything_diffusion.models.flow_diffuser_dgdit import (
    FlowTrajectoryDiffuserSimulationModule_DGDiT,
)
network = DGDiT(
    in_channels=3,
    depth=5,
    hidden_size=128,
    patch_size=1,
    num_heads=4,
    n_points=cfg.dataset.n_points,
).cuda()
model = FlowTrajectoryDiffuserSimulationModule_DGDiT(
    network, inference_cfg=cfg.inference, model_cfg=cfg.model
).cuda()
model.eval()

ckpt = torch.load('/home/yishu/open_anything_diffusion/logs/train_trajectory_diffuser_dgdit/2024-03-10/10-57-58/checkpoints/epoch=364-step=63875-val_loss=0.00-weights-only.ckpt')
network.load_state_dict(
    {k.partition(".")[2]: v for k, v, in ckpt["state_dict"].items()}
)

In [ ]:
# from open_anything_diffusion.models.modules.dit_models import DiT
# from open_anything_diffusion.models.flow_diffuser_dit import (
#     FlowTrajectoryDiffuserSimulationModule_DiT,
# )
# network = DiT(
#     in_channels=3 + 3,
#     depth=5,
#     hidden_size=128,
#     num_heads=4,
#     learn_sigma=True,
# ).cuda()
# model = FlowTrajectoryDiffuserSimulationModule_DiT(
#     network, inference_cfg=cfg.inference, model_cfg=cfg.model
# ).cuda()
# model.eval()

# ckpt = torch.load('/home/yishu/open_anything_diffusion/logs/train_trajectory_diffuser_dit/2024-03-10/10-54-13/checkpoints/epoch=459-step=80500-val_loss=0.00-weights-only.ckpt')
# network.load_state_dict(
#     {k.partition(".")[2]: v for k, v, in ckpt["state_dict"].items()}
# )

In [ ]:
# import rpad.pyg.nets.pointnet2 as pnp
# from open_anything_diffusion.models.flow_trajectory_predictor import FlowSimulationInferenceModule
# network = pnp.PN2Dense(
#     in_channels=0,
#     out_channels=3 * cfg.inference.trajectory_len,
#     p=pnp.PN2DenseParams(),
# )
# ckpt = torch.load('/home/yishu/open_anything_diffusion/logs/train_trajectory_pn++/2024-03-18/12-13-31/checkpoints/epoch=78-step=3476-val_loss=0.00-weights-only.ckpt')
# network.load_state_dict(
#     {k.partition(".")[2]: v for k, v, in ckpt["state_dict"].items()}
# )
# model = FlowSimulationInferenceModule(
#     network, False
# )
# model.eval()

In [ ]:
door_ids = ["8867", "8983", "8994", "9003", "9263", "9393"]
obj_id = door_ids[-1]

In [ ]:
import os
from rpad.partnet_mobility_utils.data import PMObject
pm_dir = os.path.expanduser("~/datasets/partnet-mobility/raw")
# env = PMSuctionSim(obj_id, pm_dir, gui=gui)
raw_data = PMObject(os.path.join(pm_dir, obj_id))

In [ ]:
available_joints = raw_data.semantics.by_type(
    "hinge"
) + raw_data.semantics.by_type("slider")
print(available_joints)

In [ ]:
available_joints = [joint.name for joint in available_joints]

In [ ]:
from open_anything_diffusion.simulations.suction import GTFlowModel, PMSuctionSim
env = PMSuctionSim(obj_id, pm_dir, gui=False)
gt_model = GTFlowModel(raw_data, env)

env.reset()

In [ ]:
target_link = available_joints[0]

In [ ]:
import pybullet as p
info = p.getJointInfo(
    env.render_env.obj_id,
    env.render_env.link_name_to_index[target_link],
    env.render_env.client_id,
)
init_angle, target_angle = info[8], info[9]
print(init_angle, target_angle)

## Initial angle

In [ ]:
env.set_joint_state(target_link, init_angle + np.linspace(0, 1, 20)[-1] * (target_angle - init_angle))
# env.set_joint_state(target_link, target_angle)
pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(P_world),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([P_world]),
    torch.as_tensor([np.zeros((1200, 3))]),
    "red",
)
fig = animation.animate()
fig.show()

## Try different open angles

In [ ]:
print(init_angle, np.linspace(init_angle, target_angle, 10), target_angle)

In [ ]:
import copy
import tqdm
from open_anything_diffusion.metrics.trajectory import flow_metrics

angle_sample_cnts = 20
trial_cnts = 20
xs = []
ys = []

for angle_ratio, angle in zip(
        np.linspace(0, 100, angle_sample_cnts),
        np.linspace(init_angle, target_angle, angle_sample_cnts)
    ):
    print(angle_ratio)
    env.set_joint_state(target_link, angle)
    # env.set_joint_state(target_link, target_angle)
    pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
    rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

    gt_flow = gt_model(pc_obs)
    nonzero_gt_flowixs = torch.where(gt_flow.norm(dim=-1) != 0.0)
    gt_flow_nz = gt_flow[nonzero_gt_flowixs]

    for i in tqdm.tqdm(range(trial_cnts)):
        with torch.no_grad():
            pred_flow = model(copy.deepcopy(pc_obs))[:, 0, :]

        pred_flow_nz = pred_flow[nonzero_gt_flowixs]

        xs.append(angle_ratio)
        # cos_dist = torch.cosine_similarity(pred_flow_nz, gt_flow_nz, dim=-1).mean()
        # ys.append(cos_dist)

        rmse, cos_dist, mag_error = flow_metrics(pred_flow_nz, gt_flow_nz)
        ys.append(mag_error)


In [ ]:
import matplotlib.pyplot as plt
plt.scatter(xs, ys)
plt.title(f'Door {obj_id}')
plt.xlabel('Open ratio (%)')
# plt.ylabel('Cosine similarity (%)')
plt.ylabel('Mag error (%)')

### Grasp point

In [ ]:
import copy
import tqdm
from open_anything_diffusion.metrics.trajectory import flow_metrics

angle_sample_cnts = 20
trial_cnts = 100
xs = []
ys = []

for angle_ratio, angle in zip(
        np.linspace(5, 100, angle_sample_cnts),
        np.linspace(init_angle, target_angle, angle_sample_cnts)
    ):
    print(angle_ratio)
    env.set_joint_state(target_link, angle)
    # env.set_joint_state(target_link, target_angle)
    pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
    rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

    gt_flow = gt_model(pc_obs)
    affordance = np.zeros_like(gt_flow)
    nonzero_gt_flowixs = torch.where(gt_flow.norm(dim=-1) != 0.0)
    gt_flow_nz = gt_flow[nonzero_gt_flowixs]

    for i in tqdm.tqdm(range(trial_cnts)):
        with torch.no_grad():
            pred_flow = model(copy.deepcopy(pc_obs))[:, 0, :]

        pred_flow_nz = pred_flow[nonzero_gt_flowixs]
        flow_norm = pred_flow.norm(p=2, dim=-1)
        max_point_id = np.argmax(flow_norm)
        affordance[max_point_id][0] += 1

        xs.append(angle_ratio)
        # cos_dist = torch.cosine_similarity(pred_flow_nz, gt_flow_nz, dim=-1).mean()
        # ys.append(cos_dist)

        rmse, cos_dist, mag_error = flow_metrics(pred_flow_nz, gt_flow_nz)
        ys.append(mag_error)

    break

In [ ]:
affordance.shape

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(P_world),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([P_world]),
    # torch.as_tensor([np.array(gt_flow)]),
    torch.as_tensor([affordance]),
    "red",
)
fig = animation.animate()
fig.show()